# Create a spark context that includes a pre-trained NLP library from JohhSnowLabs

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.0.0") \
    .getOrCreate()
spark

:: loading settings :: url = jar:file:/home/ubuntu/anaconda3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-839297e2-517c-4836-8af6-93346c75b64a;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.0.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failur

	[SUCCESSFUL ] io.grpc#grpc-protobuf;1.53.0!grpc-protobuf.jar (2ms)
downloading https://repo1.maven.org/maven2/io/grpc/grpc-protobuf-lite/1.53.0/grpc-protobuf-lite-1.53.0.jar ...
	[SUCCESSFUL ] io.grpc#grpc-protobuf-lite;1.53.0!grpc-protobuf-lite.jar (10ms)
downloading https://repo1.maven.org/maven2/io/grpc/grpc-core/1.53.0/grpc-core-1.53.0.jar ...
	[SUCCESSFUL ] io.grpc#grpc-core;1.53.0!grpc-core.jar (5ms)
downloading https://repo1.maven.org/maven2/com/google/api/gax/2.23.2/gax-2.23.2.jar ...
	[SUCCESSFUL ] com.google.api#gax;2.23.2!gax.jar (4ms)
downloading https://repo1.maven.org/maven2/com/google/api/gax-grpc/2.23.2/gax-grpc-2.23.2.jar ...
	[SUCCESSFUL ] com.google.api#gax-grpc;2.23.2!gax-grpc.jar (3ms)
downloading https://repo1.maven.org/maven2/com/google/auth/google-auth-library-credentials/1.16.0/google-auth-library-credentials-1.16.0.jar ...
	[SUCCESSFUL ] com.google.auth#google-auth-library-credentials;1.16.0!google-auth-library-credentials.jar (3ms)
downloading https://repo1.

	67 artifacts copied, 5 already retrieved (668658kB/2932ms)
23/07/18 05:53:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/18 05:53:27 ERROR Inbox: Ignoring error
java.lang.NullPointerException
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$register(BlockManagerMasterEndpoint.scala:534)
	at org.apache.spark.storage.BlockManagerMasterEndpoint$$anonfun$receiveAndReply$1.applyOrElse(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:103)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100)
	at org.apache.spark.

# Read multiple JSON files from Reddit as DataFrame

In [2]:
df = spark.read.json("./reddit/*.json")
print(f'Number of rows: {df.count()}')
df.printSchema()

23/07/18 05:53:38 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Number of rows: 100
root
 |-- data: struct (nullable = true)
 |    |-- approved_at_utc: string (nullable = true)
 |    |-- approved_by: string (nullable = true)
 |    |-- archived: boolean (nullable = true)
 |    |-- author: string (nullable = true)
 |    |-- author_flair_background_color: string (nullable = true)
 |    |-- author_flair_css_class: string (nullable = true)
 |    |-- author_flair_richtext: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- author_flair_template_id: string (nullable = true)
 |    |-- author_flair_text: string (nullable = true)
 |    |-- author_flair_text_color: string (nullable = true)
 |    |-- author_flair_type: string (nullable = true)
 |    |-- author_fullname: string (nullable = true)
 |    |-- author_patreon_flair: boolean (nullable = true)
 |    |-- banned_at_utc: string (nullable = true)
 |    |-- banned_by: string (nullable = true)
 |    |-- can_gild: boolean (nullable = true)
 |    |-- can_mod_post: boolean (n

# You can query nested subfields in JSON by using attributes: "main_field.subfield"

In [3]:
df_author_title = df.select("data.title", "data.author")
df_author_title.limit(5).toPandas()

,title,author
0,Microsoft Corp said it has discovered hacking ...,jaykirsch
1,Deutsche Bank reportedly planned to extend the...,canuck_burger
2,"Iranian ""morality police"" were forced to fire ...",honolulu_oahu_mod
3,Trump administration 'pushing Saudi nuclear de...,madam1
4,"NASA Happily Reports the Earth is Greener, Wit...",purplexxx


# Implement a simple word count using split by space and exploding an array (putting it into rows)

In [4]:
import pyspark.sql.functions as F

# Split title by " "
df_split = df.select(F.split("data.title", ' ').alias("word_list"))
df_split.limit(5).toPandas()

,word_list
0,"[Microsoft, Corp, said, it, has, discovered, h..."
1,"[Deutsche, Bank, reportedly, planned, to, exte..."
2,"[Iranian, ""morality, police"", were, forced, to..."
3,"[Trump, administration, 'pushing, Saudi, nucle..."
4,"[NASA, Happily, Reports, the, Earth, is, Green..."


In [5]:
# Explode the array into rows and count the words
df_explode = df_split.select(F.explode("word_list").alias("word")).groupBy("word").count().orderBy(F.desc("count"))
df_explode.limit(5).toPandas()

,word,count
0,to,58
1,the,46
2,of,42
3,in,41
4,a,25


# Because this word count is simple and it contains prepositions like "to", "the" etc. We need something more powerful. Hence, JohnSnowLabs pretrained NLP pipeline.

In [18]:
# Import pretrained pipeline
from sparknlp.pretrained import PretrainedPipeline
explain_document_pipeline = PretrainedPipeline("explain_document_ml")

# annotate is the basic result of the pipeline
result = explain_document_pipeline.model.transform(df_author_title.toDF('text'))
result

explain_document_ml download started this may take some time.
Approx size to download 9 MB
[OK!]


IllegalArgumentException: requirement failed: The number of columns doesn't match.
Old column names (2): title, author
New column names (1): text

In [19]:
# TODO Debug PretrainedPipeline call